In [1]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from EnergySaverLLM.Agent import ChargingAgent

In [14]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-3.5-turbo"],
        # "model": ["gpt-3.5-turbo"],
    },
)

In [3]:
autogen.oai.ChatCompletion.start_logging()

In [4]:
code_path = "EnergySaverLLM/Model/EVCharging.py"

In [5]:
with open(code_path) as f:
    code = f.read()

In [6]:
# global params_filepath
params_filepath = "EnergySaverLLM/Model/params/EVCharging.json"

In [7]:
with open(params_filepath) as f:
    json = f.read()

In [8]:
# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

from datetime import datetime
from gurobipy import GRB, Model
import json

# Fetching data from JSON file

params_filepath = "EnergySaverLLM/Model/params/EVCharging.json"

stored_params = json.load(open(params_filepath))

.
.
.

    values = model.getAttr("X", all_vars)
    names = model.getAttr("VarName", all_vars)

    for i, time_str in enumerate(keys):
        if(values[i]>0):
            print(f"Scheduled consumption at Hour {time_str} : {int(values[i]):d} KWH")

else:
    print("Not solved to optimality. Optimization status:", model.status)



In [9]:
example_qa = """
----------
Instruction: Don't charge the car from 8 PM to 10 PM
Answer Code:
```python
upper_bounds[20:22] = [0]*(22-20)
```

----------
Question: Electricity costs are have been increased by 30%, adjust the schedule accordingly
Answer Code:
```python
elec_costs.update((x, y*1.3) for x, y in elec_costs.items())
```
"""

In [15]:
%%capture
agent = ChargingAgent(
    name="Tesla Charging Example",
    source_code=code,
    debug_times=1,
    example_qa=example_qa,
    json=json,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [16]:
user.initiate_chat(agent, message="Turn charging off between 12 AM and 4 AM tonight")

user (to Tesla Charging Example):

Turn charging off between 12 AM and 4 AM tonight

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
upper_bounds[24:28] = [0] * (28 - 24)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
upper_bounds[24:28] = [0] * (28 - 24)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
{'0': 0.3, '1': 0.3, '2': 0.3, '3': 0.3, '4': 0.3, '5': 0.3, '6': 0.3, '7': 0.35, '8': 0.35, '9': 0.35, '10': 0.35, '11': 0.3, '12': 0.3, '13': 0.3, '14': 0.3, '15': 0.3, '16': 0.3, '17'

In [17]:
user.initiate_chat(agent, message="Reduce max allowed electricity cost to 15 dollars")

user (to Tesla Charging Example):

Reduce max allowed electricity cost to 15 dollars

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```python
max_elec_cost = 15
model.addConstr(sum(x[i] * elec_costs[i] for i in keys) <= max_elec_cost)
```

--------------------------------------------------------------------------------
Tesla Charging Example (to safeguard):


--- Code ---
max_elec_cost = 15
model.addConstr(sum(x[i] * elec_costs[i] for i in keys) <= max_elec_cost)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to Tesla Charging Example):

SAFE

--------------------------------------------------------------------------------
{'0': 0.3, '1': 0.3, '2': 0.3, '3': 0.3, '4': 0.3, '5': 0.3, '6': 0.3, '7':